In [ ]:
# pip install librosa
# pip install keras
# pip install tensorflow
# pip install os

In [ ]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import load_model

import module.lstm as LSTM_MODEL
import module.preprocess as PROCESSING

In [ ]:
encoder = LabelEncoder()
FEATURES = "MFCC20_ALL"
ROOT = "D:/DATA"
SR = 16000
TARGET_EPOCH = 0
PLUS_EPOCH = 10


TRAIN_PATH = f"{ROOT}/train"
VAL_PATH = f"{ROOT}/val"
TENSER_PATH = f"{ROOT}/tensor/{FEATURES}"
CHECKPOINT_PATH = f"{ROOT}/checkpoint"
CHECK_POINT_FILE = f"{FEATURES}/checkpoint{TARGET_EPOCH}.h5"
REPLACE_FILE = f"{FEATURES}/checkpoint{(TARGET_EPOCH+PLUS_EPOCH)}.h5"

In [ ]:
#wav파일들을 npy파일들로 변환하기
# X, y = PROCESSING.createDATA("D:/ToneMate_wav",SR,20,True,True,True,True,True)

In [ ]:
# npy모두 합쳐서 학습데이터 생성하기
X = []
y = []
for root, dirs, files in os.walk("D:/ToneMate_numpy"):
    for file in files:
        if file.endswith(".npy"):
            label = file.split(" - ")
            singer = label[0]
            title = label[1][:-4]
            path = os.path.join(root, file) 
            data = np.load(path)
            if(data.shape[0]==36 and data.shape[1]==3000):
                X.append(data)
                y.append(singer)
X = np.stack(X)
y = np.array(y)
if not os.path.exists(TENSER_PATH):
    os.makedirs(TENSER_PATH)

np.save(f'{TENSER_PATH}/X.npy', X)
np.save(f'{TENSER_PATH}/y.npy', y)



In [ ]:
X, y = np.load(f'{TENSER_PATH}/X.npy'), np.load(f'{TENSER_PATH}/y.npy')
X = np.transpose(X, (0, 2, 1))
encoder = encoder.fit(y)
X_train, X_valid, y_train, y_valid = train_test_split(X, encoder.transform(y), test_size=0.2, random_state=42)

In [ ]:
model = LSTM_MODEL.create_lstm_model(len(encoder.classes_),sr = SR,feature_size=X.shape[2])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
checkpoint_callback = LSTM_MODEL.Checkpoint(f"{CHECKPOINT_PATH}/{CHECK_POINT_FILE}")


if os.path.isfile(f"{CHECKPOINT_PATH}/{CHECK_POINT_FILE}"):
    model = load_model(f"{CHECKPOINT_PATH}/{CHECK_POINT_FILE}")
    #model.load_weights(f"{CHECKPOINT_PATH}/{CHECK_POINT_FILE}")


In [ ]:
if not os.path.exists(f"{CHECKPOINT_PATH}/{FEATURES}"):
    os.makedirs(f"{CHECKPOINT_PATH}/{FEATURES}")

model.fit(X_train, to_categorical(y_train,num_classes=len(encoder.classes_)), epochs= (TARGET_EPOCH+PLUS_EPOCH), validation_data=(X_valid, to_categorical(y_valid,num_classes=len(encoder.classes_))), callbacks=[checkpoint_callback], initial_epoch=TARGET_EPOCH)

In [ ]:
# pred = model.predict(test_wav)
# top_5 = np.argsort(pred, axis=1)[:, -5:]
# top_5_labels = encoder.inverse_transform(top_5.T.ravel())

In [34]:
import os
import numpy as np
singer_g = []
title_g = []
mean_g = []
var_g = []
NUMPY_PATH = f"C:/Users/wjdgm/Desktop/SAMPLE"

for root, dirs, files in os.walk(NUMPY_PATH):
    for file in files:
        if file.endswith(".npy"):
            label = file.split(" - ")
            singer = label[0]
            title = label[1][:-4]
            path = os.path.join(root, file) 
            data = np.load(path)
            data_mean = [round(attr, 5) for attr in data.mean(axis=1)]
            data_var = [round(attr, 5) for attr in data.var(axis=1)]

            singer_g.append(singer)
            title_g.append(title)
            mean_g.append(data_mean)
            var_g.append(data_var)
            print(singer,":",title)
            
mean_g = np.array(mean_g)
var_g = np.array(var_g)



#표준화
mean_g_mean_by_col = mean_g.mean(axis=0)
mean_g_std_by_col = mean_g.std(axis=0)
var_g_mean_by_col = var_g.mean(axis=0)
var_g_std_by_col = var_g.std(axis=0)

mean_g = (mean_g-mean_g_mean_by_col)/mean_g_std_by_col
var_g = (var_g-var_g_mean_by_col)/var_g_std_by_col

YB : A Cactus
YB : A Flying Butterfly
ZionT : 5월의 밤 (May)
ZionT : Comedian
ZionT : Eat


In [35]:
mean_g = (mean_g-mean_g_mean_by_col)/mean_g_std_by_col
var_g = (var_g-var_g_mean_by_col)/var_g_std_by_col